In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from tqdm.notebook import tqdm
import warnings

plt.rc('font', family='Apple SD Gothic Neo')

warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
df = pd.read_excel('./data/서울_범죄발생_지역_20220519115305.xlsx')
df.head(10)

,범죄별(1),범죄별(2),범죄별(3),2018,2019,2020
0,범죄별(1),범죄별(2),범죄별(3),서울,서울,서울
1,범죄발생총건수(A) (건),소계,소계,341288.0,340504.0,318320.0
2,인구(B) (명),소계,소계,9765623.0,9729107.0,9668465.0
3,"A/Bⅹ100,000 (건/10만명)",소계,소계,3494.8,3499.8,3292.4
4,형법범 (건),소계,소계,213505.0,220375.0,212043.0
5,형법범 (건),재산범죄,소계,125729.0,133564.0,134003.0
6,형법범 (건),재산범죄,절도,39306.0,42362.0,38414.0
7,형법범 (건),재산범죄,장물,194.0,164.0,67.0
8,형법범 (건),재산범죄,사기,57121.0,61178.0,66999.0
9,형법범 (건),재산범죄,횡령,16659.0,17648.0,16726.0


In [4]:
df = df.loc[5:]

In [5]:
# 특별범법은 제외
df = df[df['범죄별(1)']=='형법범 (건)']
df.tail()

,범죄별(1),범죄별(2),범죄별(3),2018,2019,2020
56,형법범 (건),기타형법범죄,공안을 해하는죄,18.0,105.0,115.0
57,형법범 (건),기타형법범죄,내란의죄,21.0,15.0,20.0
58,형법범 (건),기타형법범죄,음용수에 관한죄,1.0,3.0,3.0
59,형법범 (건),기타형법범죄,일수와 수리에 관한죄,-,-,-
60,형법범 (건),기타형법범죄,기타,14.0,34.0,14.0


In [6]:
df['범죄별(2)'].unique()

array(['재산범죄', '강력범죄(흉악)', '강력범죄(폭력)', '위조범죄', '공무원범죄', '풍속범죄', '과실범죄',
       '기타형법범죄'], dtype=object)

- 범죄별(2)에 따른 시각화
- 범죄별(2) 각각에 대한 볌죄별(3) 시각화


# 범죄별(2)에 따른 시각화

## 반응형 시각화 library: Pyechart

In [7]:
!pip install chart_studio
!pip install pyecharts
!pip install echarts-themes-pypkg

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=1f2890ce972f62fa03c2255e53fabd641dcf53315d49710ba74ac650ef9bcbd6
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ce\18\7f\e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built retrying
  Using cached pyecharts_jupyter_installer-0.0.3-py2.py3-none-any.whl (4.7 kB)
  Created wheel for echarts-themes-pypkg: filename=echarts_themes_pypkg-0.0.3-py3-none-any.whl size=18886 sha256=8d000a02120e114076fe5dc0efa498415462dfc6fd36654ad103e5dcd785fc7e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\41\82\a5\e48922513ec9ab54c183122decbd184e106d7c85ce84bcd8eb
Successfully built echarts-themes-pypkg


In [9]:
!pip install folium

In [10]:
import pandas as pd
from pandas import Series
import numpy as np
from datetime import date

import copy, json
import chart_studio
chart_studio.tools.set_credentials_file(username='username', api_key='api_key')

from IPython.display import IFrame
import chart_studio.plotly as py
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.io as pio

from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Bar, Bar3D, Grid, Line, Liquid, Page, Pie, Timeline, Tab, Map, Grid
from pyecharts import options as opts

import folium, webbrowser, IPython

In [11]:
big_crime = df['범죄별(2)'].unique()
print(big_crime)

['재산범죄' '강력범죄(흉악)' '강력범죄(폭력)' '위조범죄' '공무원범죄' '풍속범죄' '과실범죄' '기타형법범죄']


In [12]:
df = df.drop('범죄별(1)', axis=1)
df

,범죄별(2),범죄별(3),2018,2019,2020
5,재산범죄,소계,125729.0,133564.0,134003.0
6,재산범죄,절도,39306.0,42362.0,38414.0
7,재산범죄,장물,194.0,164.0,67.0
8,재산범죄,사기,57121.0,61178.0,66999.0
9,재산범죄,횡령,16659.0,17648.0,16726.0
10,재산범죄,배임,1536.0,1489.0,1283.0
11,재산범죄,손괴,10913.0,10723.0,10514.0
12,강력범죄(흉악),소계,9975.0,9939.0,8814.0
13,강력범죄(흉악),살인,150.0,160.0,168.0
14,강력범죄(흉악),강도,145.0,157.0,143.0


In [13]:
df_crime_total = df[df['범죄별(3)']=='소계']
df_crime_total

,범죄별(2),범죄별(3),2018,2019,2020
5,재산범죄,소계,125729.0,133564.0,134003.0
12,강력범죄(흉악),소계,9975.0,9939.0,8814.0
17,강력범죄(폭력),소계,45815.0,44745.0,38551.0
26,위조범죄,소계,4049.0,4121.0,3432.0
31,공무원범죄,소계,1270.0,1157.0,961.0
36,풍속범죄,소계,1480.0,1952.0,2075.0
40,과실범죄,소계,1497.0,1567.0,1494.0
44,기타형법범죄,소계,23690.0,23330.0,22713.0


In [14]:
df_crime_total.columns[2:].tolist()

['2018', '2019', '2020']

In [15]:
# 범죄별(2)의 통계에 대한 연도별 꺾은선 그래프

df_crime_total = df[df['범죄별(3)']=='소계']
df_crime_melt = pd.melt(df_crime_total, id_vars=['범죄별(2)', '범죄별(3)'])

fig_gender = (Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS, height='600px',width='800px'))
       .add_xaxis(df_crime_total.columns[2:].tolist())
       .add_yaxis('재산범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='재산범죄']['value'].tolist())
       .add_yaxis('강력범죄(흉악)', df_crime_melt[df_crime_melt['범죄별(2)']=='강력범죄(흉악)']['value'].tolist())
       .add_yaxis('강력범죄(폭력)', df_crime_melt[df_crime_melt['범죄별(2)']=='강력범죄(폭력)']['value'].tolist())
       .add_yaxis('위조범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='위조범죄']['value'].tolist())
       .add_yaxis('공무원범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='공무원범죄']['value'].tolist())
       .add_yaxis('풍속범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='풍속범죄']['value'].tolist())
       .add_yaxis('과실범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='과실범죄']['value'].tolist())
       .add_yaxis('기타형범범죄', df_crime_melt[df_crime_melt['범죄별(2)']=='기타형범범죄']['value'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title='부산광역시 범죄 유형에 따른 발생 건수 변화',
                                                                  subtitle='마우스를 line에 올려 연도별 범죄 유형 발생 건수를 확인해보세요!'),
                        xaxis_opts=opts.AxisOpts(name='연도'),
                        yaxis_opts=opts.AxisOpts(name='범죄 발생 총 건수', min_=0, name_location="start"),
                        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
                        legend_opts=opts.LegendOpts(pos_top='9%',pos_left='10%',legend_icon='pin')))

fig_gender.render_notebook()

In [32]:
fig_household_tab = Tab()

# 1. 재산범죄
# 파이차트

# 범죄별(2) 유형에 따른 pie 그래프 생성 및 추가

for i, crime in enumerate(big_crime):
    tmp_df = df[df['범죄별(2)']==crime].iloc[1:]
    globals()['fig_pie_{}'.format(i)] =  Timeline(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS, height='600px',width='800px')).add_schema(is_auto_play=True)

    for t in ['2018', '2019', '2020']:
        globals()['fig_pie_{}'.format(i)].add((Pie(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS)).add('', [list(z) for z in zip(tmp_df['범죄별(3)'].values.tolist(),
                                                         tmp_df[t].values.tolist())],
                               label_opts=opts.LabelOpts(position='outside', formatter="{b}\n{d}%"),
                               radius=[70, 180])
                               .set_global_opts(title_opts=opts.TitleOpts(title='연도별 서울특별시 ' + '{}'.format(crime) + ' 유형',
                                                                          subtitle='마우스를 파이에 올려 유형별 건수를 확인해보세요!'),
                               legend_opts=opts.LegendOpts(type_ = 'scroll', pos_bottom ="8%",pos_left="0%", 
                                                           orient="vertical",legend_icon ='pin')))
                                                           ,"{}".format(t))

    fig_household_tab.add(globals()['fig_pie_{}'.format(i)], '서울특별시 ' + '{}'.format(crime))

fig_household_tab.render_notebook()